In [26]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models
import random
from cnn_forward import *

seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 145s 1us/step


In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

In [9]:
x_train = x_train/255.0
x_val = x_val/255.0
x_test = x_test/255.0

In [14]:
model = models.Sequential([
    layers.Conv2D(10, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()

model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data=(x_val, y_val))

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 30, 30, 10)     │           280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 15, 15, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 2250)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │        22,510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,790 (89.02 KB)

 Trainable params: 22,790 (89.02 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.3212 - loss: 1.9107 - val_accuracy: 0.4698 - val_loss: 1.5181
Epoch 2/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4859 - loss: 1.4806 - val_accuracy: 0.5056 - val_loss: 1.4104
Epoch 3/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5212 - loss: 1.3763 - val_accuracy: 0.5346 - val_loss: 1.3363
Epoch 4/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5502 - loss: 1.2984 - val_accuracy: 0.5488 - val_loss: 1.2828
Epoch 5/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5677 - loss: 1.2413 - val_accuracy: 0.5640 - val_loss: 1.2467
Epoch 6/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5816 - loss: 1.2008 - val_accuracy: 0.5684 - val_loss: 1.2233
Epoch 7/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5917 - loss: 1.1705 - val_accuracy: 0.5764 - val_loss: 1.2075
Epoch 8/10
704/704 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5999 - loss: 1.1464 - val_accuracy: 0.

In [16]:
y_pred_tensorflow=model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [ ]:
y_pred_tensorflow=np.argmax(y_pred_tensorflow, axis=1) # Hasil klasifikasi dalam integer

In [21]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_pred_tensorflow, average='macro')
print("F1 Score (Macro):", f1)

F1 Score (Macro): 0.5859761330968082


In [25]:
# implementasi from scratch
W_conv = np.array(model.get_weights()[0])
b_conv = np.array(model.get_weights()[1])

W_dense = np.array(model.get_weights()[2])
b_dense = np.array(model.get_weights()[3])

In [27]:
def predict_scratch(X, w_conv, b_conv, w_dense, b_dense):
    conv_out = conv2d_forward(x_test, W_conv, b_conv)
    pooled = max_pooling2d_forward(conv_out)
    flattened = flatten(pooled)
    dense_output = dense_forward(flattened, W_dense, b_dense, softmax)
    return np.argmax(dense_output, axis=1)    

In [29]:
y_scratch = predict_scratch(x_test, W_conv, b_conv, W_dense, b_dense)

100%|██████████| 10000/10000 [01:26<00:00, 115.82it/s]


In [31]:
from sklearn.metrics import f1_score

f1 = f1_score(y_test, y_scratch, average='macro')
print("F1 Score (Macro):", f1)

F1 Score (Macro): 0.5859761330968082
